In [ ]:
import torch

torch.cuda.is_available()

In [ ]:
print(good_score[0][:, 1] > bad_score[0][:, 1]) # True, label mapping: "0" -> "Hallucinated" "1" -> "Faithful"

In [ ]:
from transformers import AutoTokenizer, GPT2ForSequenceClassification
tokenizer_tristan = AutoTokenizer.from_pretrained("Tristan/gpt2_reward_summarization")

In [ ]:
input_ids

bad tensor([0.5631])
good tensor([0.5590])


In [ ]:
import numpy as np

In [ ]:
input_ids

In [ ]:
model??

In [ ]:
bad_pair = tokenizer(text=bad_summary, text_pair=article, return_tensors='pt')
good_pair = tokenizer(text=good_summary, text_pair=article, return_tensors='pt')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("Tristan/gpt2_reward_summarization")

bad_score = model(**bad_pair)
good_score = model(**good_pair)

In [ ]:
bad_score[0]

In [ ]:
good_score[0]

In [ ]:
tokenizer_tristan??

In [ ]:
tokenizer_tristan.decode(good_pair["input_ids"][0])

In [ ]:
tokenizer.decode(inputs["input_ids"][0])

In [ ]:
model = GPT2ForSequenceClassification.from_pretrained("sugam11/gpt2-rlhf-reward")


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
reward_name = "OpenAssistant/reward-model-electra-large-discriminator"
rank_model, tokenizer = AutoModelForSequenceClassification.from_pretrained(reward_name), AutoTokenizer.from_pretrained(reward_name)
question, answer = "Explain nuclear fusion like I am five", "Nuclear fusion is the process by which two or more protons and neutrons combine to form a single nucleus. It is a very important process in the universe, as it is the source of energy for stars and galaxies. Nuclear fusion is also a key process in the production of energy for nuclear power plants."
inputs = tokenizer(question, answer, return_tensors='pt')

In [ ]:
tokenizer.decode(inputs["input_ids"][0])

In [ ]:
# coding=utf-8
# Copyright 2022 The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from datetime import datetime

from tqdm import tqdm
from transformers import LlamaForCausalLM, LlamaTokenizer, HfArgumentParser, pipeline
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, PeftModel


########################################################################
# This is a fully working simple example to use trl with accelerate.
#
# This example fine-tunes a GPT2 model on the IMDB dataset using PPO
# (proximal policy optimization).
# in any of the following settings (with the same script):
#   - single CPU or single GPU
#   - fp16 (mixed-precision) or fp32 (normal precision)
#
# To run it in each of these various modes, first initialize the accelerate
# configuration with `accelerate config`
#
########################################################################

########################################################################
# NOTE for to train with a 8-bit model a more recent version of
# transformers is required, full dependecies for this example:
# pip install  bitsandbytes datasets accelerate loralib
# pip install  git+https://github.com/huggingface/transformers.git@main
# pip install peft
########################################################################

# If you want to log with tensorboard, add the kwarg
# `accelerator_kwargs={"logging_dir": PATH_TO_LOGS}` to the PPOConfig.


class DefaultArgs:
    learning_rate = 1.41e-5
    sentiment_model = "lvwerra/distilbert-imdb"
    model_name = "decapoda-research/llama-7b-hf"
    lora_config = "tloen/alpaca-lora-7b"
    log_with = ""
    logs = ""
    prompt = "medium"
    score_goal = "1"
    mini_batch_size = 4
    batch_size = 128
    gradient_accumulation_steps = 1
    init_kl_coef = 0.2
    adap_kl_ctrl = 1
    ref_model = None
    output_min_length = 18

@dataclass
class ScriptArguments:
    """
    The name of the Casual LM model we wish to fine with PPO
    """

    model_name: Optional[str] = field(
        default=DefaultArgs.model_name, metadata={"help": "the model name"}
    )
    # decapoda-research/llama-13b-hf
    #
    lora_config: Optional[str] = field(
        default=DefaultArgs.lora_config, metadata={"help": "the lora config"}
    )
    # chansung/alpaca-lora-13b
    log_with: Optional[str] = field(
        default='wandb', metadata={"help": "use 'wandb' to log with wandb"}
    )
    logs: Optional[str] = field(default=DefaultArgs.logs, metadata={"help": "logs"})
    prompt: Optional[str] = field(default=DefaultArgs.prompt, metadata={"help": "prompt"})
    learning_rate: Optional[float] = field(
        default=DefaultArgs.learning_rate, metadata={"help": "the learning rate"}
    )

    sentiment_model: Optional[str] = field(
        default=DefaultArgs.sentiment_model, metadata={"help": "which sentiment model to use"}
    )

    score_goal: Optional[str] = field(
        default=DefaultArgs.score_goal, metadata={"help": "which sentiment to use"}
    )
    mini_batch_size: Optional[int] = field(
        default=DefaultArgs.mini_batch_size, metadata={"help": "the PPO minibatch size"})
    batch_size: Optional[int] = field(
        default=DefaultArgs.batch_size, metadata={"help": "the batch size"})
    init_kl_coef: Optional[float] = field(
        default=DefaultArgs.init_kl_coef, metadata={"help": "init_kl_coef"})
    adap_kl_ctrl: Optional[int] = field(
        default=DefaultArgs.adap_kl_ctrl, metadata={"help": "adap_kl_ctrl"}
    )
    output_min_length: Optional[int] = field(
        default=DefaultArgs.output_min_length, metadata={"help": "output_min_length"})
    ref_model: Optional[str] = field(
        default=DefaultArgs.ref_model, metadata={"help": "the ref_model. Can be None or copy"}
    )
    gradient_accumulation_steps: Optional[int] = field(
        default=DefaultArgs.gradient_accumulation_steps, metadata={"help": "the number of gradient accumulation steps"}
    )


class Loader:

    @staticmethod
    def load_base_model(base_model_name):
        base_model = LlamaForCausalLM.from_pretrained(
            base_model_name, load_in_8bit=True, device_map="auto"
        )
        base_model = prepare_model_for_int8_training(base_model)
        return base_model

    @staticmethod
    def load_peft_model(base_model, lora_config):
        """### Apply LoRA
        Here comes the magic with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.
        """
        if lora_config in [None, "none"]:
            lora_config_loaded = LoraConfig(
                r=8,
                lora_alpha=16,
                lora_dropout=0.05,
                target_modules=None,
                bias="none",
                task_type="CAUSAL_LM",
            )
            model = get_peft_model(base_model, lora_config_loaded)
            lora_model = model
        elif lora_config == "nolora":
            raise ValueError()
            # model = base_model
            # lora_model = PeftModel.from_pretrained(model, DefaultArgs.lora_config)
        else:
            model = PeftModel.from_pretrained(base_model, lora_config)
            lora_model = model
        modelvaluehead = AutoModelForCausalLMWithValueHead.from_pretrained(model)
        return modelvaluehead, lora_model

    @staticmethod
    def print_trainable_parameters(model):
        """
        Prints the number of trainable parameters in the model.
        """
        trainable_params = 0
        all_param = 0
        for _, param in model.named_parameters():
            all_param += param.numel()
            if param.requires_grad:
                trainable_params += param.numel()
        print(
            f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
        )


class Tokenizer:
    @staticmethod
    def load_tokenizer(base_model_name):
        tokenizer = LlamaTokenizer.from_pretrained(base_model_name, add_eos_token=True)
        tokenizer.pad_token_id = 0
        tokenizer.padding_side = "left"
        return tokenizer



if False:
    parser = HfArgumentParser(ScriptArguments)
    script_args = parser.parse_args_into_dataclasses()[0]
else:
    script_args = DefaultArgs()

config = PPOConfig(
    model_name=script_args.model_name,
    init_kl_coef=script_args.init_kl_coef,
    adap_kl_ctrl=script_args.adap_kl_ctrl,
    learning_rate=script_args.learning_rate,
    log_with=script_args.log_with,
    batch_size=script_args.batch_size,
    mini_batch_size=script_args.mini_batch_size,
    optimize_cuda_cache=True,
    gradient_accumulation_steps=script_args.gradient_accumulation_steps,
)

# set seed before initializing value head for deterministic eval
set_seed(config.seed)


base_model = Loader.load_base_model(script_args.model_name)
model, _ = Loader.load_peft_model(base_model, lora_config=script_args.lora_config)
tokenizer = Tokenizer.load_tokenizer(script_args.model_name)
Loader.print_trainable_parameters(model)

class Instructions:
    instructions_summary = [
        "Read the following text and generate a 1 short sentence summary.",
        "Write a concise summary of a fact.", "Write a concise summary of the provided text.",
        "Create a summary of the text below", "Generate a summary.",
        "Write a one-sentence summary of the following news article.",
        "Construct a concise summary of the following article.",
        "Generate a one-sentence summary of the story.",
        "Analyze the following news article and provide a brief summary.",
        "Write a summary of the given article. Make sure it is less than 100 words and it accurately reflects the key points of the article."
    ]
    instruction_movie = "Generate a movie review."

    if script_args.prompt == "long":
        prompt_movie = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
            ### Instruction: {Instructions.instruction_movie}
            ### Response: """
    elif script_args.prompt == "hhh":
        prompt_movie = "### Instruction: Generate an helpful harmless honest movie review. ### Response: "
    elif script_args.prompt == "medium":
        prompt_movie = "### Instruction: Generate a movie review. ### Response: "
    else:
        prompt_movie = "Instruction: Generate a movie review. Response: "
    size_prompt_movie = len(tokenizer.encode(prompt_movie)) - 1

# Below is an example function to build the dataset. In our case, we use the IMDB dataset
# from the `datasets` library. One should customize this function to train the model on
# its own dataset.
def build_dataset(
    config, tokenizer, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8
):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    # load imdb with datasets
    ds = load_dataset(dataset_name, split="train")
    ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

    input_size = LengthSampler(
        Instructions.size_prompt_movie + input_min_text_length,
        Instructions.size_prompt_movie + input_max_text_length
    )

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(Instructions.prompt_movie +
                                               sample["review"])[:input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

# We retrieve the dataloader by calling the `build_dataset` function.
dataset = build_dataset(config, tokenizer=tokenizer)
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate
)

# We then build the PPOTrainer, passing the model, the reference model, the tokenizer


In [ ]:
if False:
    parser = HfArgumentParser(ScriptArguments)
    script_args = parser.parse_args_into_dataclasses()[0]
else:
    script_args = NotebookArgs()

In [ ]:
base_model = Loader.load_base_model(script_args.model_name)

In [ ]:
base_model.is_peft_model

In [ ]:
model, lora_model = Loader.load_peft_model(base_model, lora_config="nolora")

In [ ]:
model.is_peft_model

In [ ]:
lora_model.is_peft_model

In [ ]:
script_args = ScriptArguments.notebook_get_args()
assert "llama" in script_args.base_model_name.lower()
print(f"Load LMs with {script_args.peft_names}")

In [ ]:
base_model = Loader.load_base_model(script_args.base_model_name)

In [ ]:
sd = base_model.state_dict()

In [ ]:
sd = base_model_v2.state_dict()

In [ ]:
sd["model.layers.2.self_attn.rotary_emb.inv_freq"]

In [ ]:
base_model_v2 = LlamaForCausalLM.from_pretrained(
            script_args.peft_names[0],     return_dict=True,
    torch_dtype=torch.float16,
        )

In [ ]:
device = 0 if torch.cuda.is_available() else "cpu"
print(f"Load LMs with {script_args.model_names}")
dict_models_to_merge = OrderedDict({model_name: load_model(model_name) for model_name in script_args.model_names})

tokenizer = AutoTokenizer.from_pretrained(
    PeftConfig.from_pretrained(script_args.model_names[0]).base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"Load sentiment model with {script_args.sentiment_models}")
sentiment_pipes = [
    pipeline("sentiment-analysis", model=sentiment_model, device=device)
    for sentiment_model in script_args.sentiment_models]

list_states_dict = []
for current_model in dict_models_to_merge.values():
    current_weights = copy.deepcopy(current_model.state_dict())
    list_states_dict.append(current_weights)

In [ ]:
samples_query_tensors = get_samples_query_tensors()

In [ ]:
wa = enrich_wa_states(list_states_dict, coefficients=[0.3, 0.7])
list_rewards_wa_samples = predict({"wa": wa}, samples_query_tensors, verbose=True)

In [ ]:
wa = enrich_wa_states(list_states_dict, coefficients=[0.25, 0.75])
list_rewards_wa_samples = predict({"wa": wa}, samples_query_tensors, verbose=True)

In [ ]:

for coeff in [x / 20 for x in range(17, -1, -1)]:
    wa = enrich_wa_states(list_states_dict, coefficients=[1 - coeff, coeff])
    list_rewards_wa_samples = predict({"wa": wa}, samples_query_tensors)
    print(coeff)
    print(list_rewards_wa_samples)
    print("\n")